# DS 3002: Data Project 1 - Charlottesville Crime ETL Process
### Catherine Schuster (cas7kvf)  -  03/21/2022

## Methodology: 
For my data project, I am interested in extracting Charlottesville crime data from the city of Charlottesville's Open Data website. [The Charlottesville Crime Data Set](https://opendata.charlottesville.org/datasets/charlottesville::crime-data/about) represents the initial information that is provided by individuals calling for police assistance in the city of Charlottesville. Remaining information is often amended for accuracy after an Officer arrives and investigates the reported incident. The data contains reports over last 5 years, and is updated nightly. 

Because the crime data is updated nightly, an optimal data extraction method would retrieve the most recent version of the data when the file is run. I met this objective by using the data's API resource URL, which contains the most updated version of data in a geojson file type. The resulting data frame is then transformed with the goal of cleaning the data (removing unecessary columns, expanding the timestamps, and recoding and consolidating criminal offense categories). The transformed data is then loaded to SQL (.geojson --> .sql) as a final destination for data storage and query use. 

## Import Libraries

In [74]:
import pymysql
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine
#!pip install geopandas
import geopandas as gpd

In [75]:
## Assign SQL User information
host_name = "localhost"
host_ip = "127.0.0.1"
port = 3306

user_id = "root"
pwd = 'Cathschus17'
dst_dbname = "Charlottesville"

## 1. Extract Data Source

In [76]:
# Retrieves real-time crime data from Charlottesville Open Data API 
# File type is .geojson
cville_crime = gpd.read_file('https://opendata.arcgis.com/datasets/d1877e350fad45d192d233d2b2600156_6.geojson')
cville_crime.head()

,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,DateReported,HourReported,ReportingOfficer,geometry
0,1,Assault Aggravated,202200006255,800,MONTICELLO RD,CPD,2022-03-16T02:01:12+00:00,2201,"Darby, Michael",None
1,2,Fraud-false pretense,202200006232,400,RIVERSIDE AVE,CPD,2022-03-15T21:25:44+00:00,1725,"Houchens, Logan",None
2,3,Pornography,202200006229,600,E MARKET ST,CPD,2022-03-15T20:50:15+00:00,1650,"Bowers, Justin",None
3,4,Pornography,202200006228,600,E MARKET ST,CPD,2022-03-15T20:50:13+00:00,1650,"Bowers, Justin",None
4,5,Pornography,202200006227,600,E MARKET ST,CPD,2022-03-15T20:50:12+00:00,1650,"Bowers, Justin",None


## 2. Transform Data

In [77]:
# Time of Day
timestamps = cville_crime["DateReported"].str.split("-", n=1, expand = True) # Split time variable

cville_crime["Year"] = timestamps[0] # Create year column 

cville_crime["Date"] = timestamps[1].str.split("T").str[0] # Create date column (month/day)
cville_crime["Date"] = cville_crime["Date"].str.replace('-', '/') # Fix date column formatting 

cville_crime["Time"] = timestamps[1].str.split("T").str[1].str.split('+').str[0] # Create time column (hour:min:sec)

cville_crime = cville_crime.drop(['DateReported', 'HourReported'], axis=1) # Drop previous columns after cleaning

cville_crime.head()

,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,ReportingOfficer,geometry,Year,Date,Time
0,1,Assault Aggravated,202200006255,800,MONTICELLO RD,CPD,"Darby, Michael",None,2022,03/16,02:01:12
1,2,Fraud-false pretense,202200006232,400,RIVERSIDE AVE,CPD,"Houchens, Logan",None,2022,03/15,21:25:44
2,3,Pornography,202200006229,600,E MARKET ST,CPD,"Bowers, Justin",None,2022,03/15,20:50:15
3,4,Pornography,202200006228,600,E MARKET ST,CPD,"Bowers, Justin",None,2022,03/15,20:50:13
4,5,Pornography,202200006227,600,E MARKET ST,CPD,"Bowers, Justin",None,2022,03/15,20:50:12


In [78]:
pd.value_counts(cville_crime['Agency']) # Angency column contains only one value (CPD - Charlottesville Polic Dept)
cville_crime = cville_crime.drop(['Agency', 'geometry'], axis=1) # Remove Agency and geometry as it is not useful data

In [79]:
# Type of Crime

# Recode offense from 115 categories to one of 7 categories 
# (drugs/alcohol, property, violent, procedural, traffic, sex, petty, other)

# Store indicator words for each category
searchfor1 = ["dwi", "dui", "drug", "drugs" "bac", "intoxication", "drunk", "drink", "alcohol", "liquor", 
              "marijuana", "mari", "narc"]
searchfor2 = ["burg", "larceny", "forgery", "fraud", "embezzlement", "property", "monument", "damage", "shop", 
              "trespass", "theft", "rob", "vandalism" "embezzelment", "extortion"]
searchfor3 = ["abduction", "assault", "strangulation", "firearm", "battery", "shoot", "armed", "murder", "homicide"
              "threat", 'stalk', 'abuse', "force", "forc", "endang", "shots fired", "arson", "child ab", "weapons",
             'homicide-negligent manslaughter']
searchfor4 = ["fta", "court", "probation", "warrant", "false report",]
searchfor5 = ["drive", "driv", "lic", "traf", "vehi", "veh", "pede", "trf", "dr", "license", "hit and run", 
              "accident -", "accident w/injuries", "traffic"]
searchfor6 = ["pornography", "sex", "harassment", "rape", "prostitution", "prostitution-assisting/promoting"]
searchfor7 = ["Traffic Crime", "Procedural Crime", "Violent Crime", "Property Crime", "Drugs/Alcohol Related", 
              "Sex Crime"]

# Assign broad categories for each type of crime
cville_crime.loc[cville_crime['Offense'].str.lower().str.contains('|'.join(searchfor1)), 'Offense'] = "Drugs/Alcohol Crime"
cville_crime.loc[cville_crime['Offense'].str.lower().str.contains('|'.join(searchfor2)), 'Offense'] = "Theft/Property Crime"
cville_crime.loc[cville_crime['Offense'].str.lower().str.contains('|'.join(searchfor3)), 'Offense'] = "Violent Crime"
cville_crime.loc[cville_crime['Offense'].str.lower().str.contains('|'.join(searchfor4)), 'Offense'] = "Procedural Crime"
cville_crime.loc[cville_crime['Offense'].str.lower().str.contains('|'.join(searchfor5)), 'Offense'] = "Traffic Crime" 
cville_crime.loc[cville_crime['Offense'].str.lower().str.contains('|'.join(searchfor6)), 'Offense'] = "Sex Crime"
cville_crime.loc[~cville_crime['Offense'].str.contains('|'.join(searchfor7)), 'Offense'] = "Other Crime"

In [80]:
# Verify offense categories
pd.value_counts(cville_crime['Offense'])

Theft/Property Crime    8343
Other Crime             8159
Traffic Crime           4359
Violent Crime           3596
Sex Crime                226
Procedural Crime         175
Name: Offense, dtype: int64

In [81]:
# Verify final dataframe
cville_crime.head()

,RecordID,Offense,IncidentID,BlockNumber,StreetName,ReportingOfficer,Year,Date,Time
0,1,Violent Crime,202200006255,800,MONTICELLO RD,"Darby, Michael",2022,03/16,02:01:12
1,2,Theft/Property Crime,202200006232,400,RIVERSIDE AVE,"Houchens, Logan",2022,03/15,21:25:44
2,3,Sex Crime,202200006229,600,E MARKET ST,"Bowers, Justin",2022,03/15,20:50:15
3,4,Sex Crime,202200006228,600,E MARKET ST,"Bowers, Justin",2022,03/15,20:50:13
4,5,Sex Crime,202200006227,600,E MARKET ST,"Bowers, Justin",2022,03/15,20:50:12


## 3. Load Data to SQL

In [82]:
# Define a function to set a database in SQL
def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [83]:
# Query to create Charlottesville database 

exec_sql = f"CREATE DATABASE `{dst_dbname}`;"
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
sqlEngine.execute(exec_sql) #create db
sqlEngine.execute("USE Charlottesville;") # select new db

In [84]:
# Set crime dataframe to a table in Charlottesville database
db_operation = "insert"
dataframe = cville_crime
table_name = "crime"
primary_key = 'RecordID'
set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

## 4. Verify SQL Database

In [86]:
# Define a function to get a database in SQL
def get_dataframe(user_id, pwd, host_ip, db_name, sql_query):
    #conn_str = 'mysql+mysqlconnector://user_name:%s@localhost:3306/database' % urlquote({pwd})
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    return dframe

In [88]:
# Retrieve crime table from Charlottesville SQL database
sql_cville_crime = "SELECT * FROM Charlottesville.crime;"
df_cville_crime = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_cville_crime)
df_cville_crime.head() 

,RecordID,Offense,IncidentID,BlockNumber,StreetName,ReportingOfficer,Year,Date,Time
0,1,Violent Crime,202200006255,800,MONTICELLO RD,"Darby, Michael",2022,03/16,02:01:12
1,2,Theft/Property Crime,202200006232,400,RIVERSIDE AVE,"Houchens, Logan",2022,03/15,21:25:44
2,3,Sex Crime,202200006229,600,E MARKET ST,"Bowers, Justin",2022,03/15,20:50:15
3,4,Sex Crime,202200006228,600,E MARKET ST,"Bowers, Justin",2022,03/15,20:50:13
4,5,Sex Crime,202200006227,600,E MARKET ST,"Bowers, Justin",2022,03/15,20:50:12


In [95]:
# Create a test query to further verify ETL process and output
test_query = "SELECT Offense, count(*) AS offense_count, \
              count(*) * 100.0/ sum(count(*)) over () AS offense_percent \
              FROM crime \
              WHERE year > 2015 \
              GROUP BY Offense \
              ORDER BY offense_percent DESC;" 

test_df = get_dataframe(user_id, pwd, host_name, dst_dbname, test_query)
test_df.head() 

,Offense,offense_count,offense_percent
0,Theft/Property Crime,8343,33.56264
1,Other Crime,8159,32.82243
2,Traffic Crime,4359,17.53560
3,Violent Crime,3596,14.46617
4,Sex Crime,226,0.90916
